# Introducción

## Descripción
Con este notebook de Python se muestra como utilizar la librería de [gensim](https://radimrehurek.com/gensim/index.html) para realizar sistemas de recuperación de información eficientes. En partícular, se muestra como cargar y procesar corpus de texto, así como utilizar representaciones vectoriales del corpus para realizar consultas basadas en similitud.

**Los objetivos de aprendizaje son**:

1. Cargar de manera eficiente datasets de texto con **gensim**.
2. Aplicar técnicas de procesamiento de texto al corpus con **gensim**.
3. Generar representaciones vectoriales del corpus con **gensim**.
4. Constuir sistemas de recuperación de información con **gensim**.

## Metodología
Este notebook será un tutorial para aprender a instalar y usar **gensim** con el fin de crear sistemas de recuperación de información robustos y eficientes. Adicionalmente, se referirá al estudiante a la documentación de la librería y de los métodos vistos para que pueda ampliar la información sobre su uso.

La estructura del notebook es la siguiente:
* [1. Corpus](#corpus)
    * [1.1 Cargar un corpus con streaming de datos para uso eficiente de la memoria](#corpus-load_data)
* [2. Procesamiento de texto](#2-procesamiento-de-texto)
    * [2.1. Otras técnicas de procesamiento de texto](#21-otras-técnicas-de-procesamiento-de-texto)
* [3. Construir el vocabulario](#3-construir-el-vocabulario)
* [4. Representaciones vectoriales](#4-representaciones-vectoriales)
    * [4.1. Representación de bolsa de palabras](#41-representación-de-bolsa-de-palabras)
    * [4.2. Representación de bolsa de palabras con puntajes **tf-idf**](#42-representación-de-bolsa-de-palabras-con-puntajes-tf-idf)
    * [4.3. Guardar los modelos entrenados y el corpus transformado](#43-guardar-los-modelos-entrenados-y-el-corpus-transformado)
    * [4.4. Compatibilidad con **numpy** y **scipy**](#44-compatibilidad-con-numpy-y-scipy)
* [5. Construcción de sistemas de recuperación](#5-construcción-de-sistemas-de-recuperación)
* [6. Conclusiones](#6-conclusiones)

# Instalación de la librería

In [1]:
# Para instalar localmente, descomente la siguiente línea
# pip install gensim

# 1. Corpus <a class="anchor" id="corpus"></a>

Para **gensim** un corpus es simplemente una colección de documentos, en donde cada documento es un string de Python con el contenido textual del documento.

A continuación vemos el ejemplo de un corpus, que será el punto de partida para desarollar nuestro sistema de recuperación de información.

In [2]:
text_corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

### 1.1 Cargar un corpus con streaming de datos para uso eficiente de la memoria <a class="anchor" id="corpus-load_data"></a>

Note que el corpus anterior reside completamente en la memoria RAM debido a que es una lista de Python. Para dataset reales, que normalmente suelen tener tamaños del orden de GBs no sería conveniente tener todo el dataset almacenado en nuestra RAM. Por esta razón, **gensim** acepta como corpus cualquier objeto iterable.

Este diseño de **gensim** nos permite en conjunto con librarías como [smart_open](https://pypi.org/project/smart-open/) (ya viene incluida con Python 3) cargar nuestro dataset sin almacenarlo directamente en RAM, sino cargar únicamente los documentos en el momento que se necesitan.

In [3]:
from smart_open import open

class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/mycorpus.txt'):
            yield line

corpus = MyCorpus()
for doc in corpus:
    print(doc)

Human machine interface for lab abc computer applications

A survey of user opinion of computer system response time

The EPS user interface management system

System and human system engineering testing of EPS

Relation of user perceived response time to error measurement

The generation of random binary unordered trees

The intersection graph of paths in trees

Graph minors IV Widths of trees and well quasi ordering

Graph minors A survey



# 2. Procesamiento de texto <a class="anchor" id="text_processing" name="text_processing"></a>

En **gensim** unsa de las formas más simples de procesar y tokenizar el texto, es a través de la función de utilidad [simple_process](https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess). Esta función se encarga de:

* Convertir el texto a minúsculas.
* Tokenizar el texto ignorando caracteres de puntuación.
* Ignorar tokens muy cortos o muy largos.
* Remover los acentos de los tokens (opcional).

In [4]:
import pprint
from gensim.utils import simple_preprocess

# Tokenizar los documentos
tokenized_docs = [simple_preprocess(doc) for doc in corpus]

# Imprimir el primer documento tokenizado
pprint.pprint(tokenized_docs[0])

['human',
 'machine',
 'interface',
 'for',
 'lab',
 'abc',
 'computer',
 'applications']


Por defecto **simple_preprocess** elimina tokens con menos de 2 caracteres o más de 15 caracteres. Además, de manera opcional se pueden remover los acentos del texto.

In [5]:
sample_text = "This, is a sample text. largewoooooooord, estás"
print(simple_preprocess(sample_text, deacc=True))

['this', 'is', 'sample', 'text', 'estas']


## 2.1 Otras técnicas de procesamiento de texto

La función **preprocess_string** de **gensim** nos permite aplicar distintas técnicas de procesamiento de texto de manera fácil.
Esta función recibe una lista de funciones de procesamiento que se aplican al texto de entrada.

Por defecto esta función aplica la sigueinte lista de filtros:
* `strip_tags()`: Elimina tags tipo HTML.
* `strip_punctuation()`: Elimina caracteres de puntuación como comas y puntos.
* `strip_multiple_whitespaces()`: Elimina multiples caracteres de espacios entre palabras.
* `strip_numeric()`: Elimina los dígitos del texto.
* `remove_stopwords()`: Elimina un listado de palabras de parada en inglés.
* `strip_short()`: Elimina palabras con menos de 3 caracteres.
* `stem_text()`: Aplicar stemming al texto. Solo funciona para inglés.

In [6]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_punctuation

sample_text = "<i>Hello9</i> <b>Wo4rld</b> apples! The     weather_is  is really good today, isn't it?"

# Aplicar el preprocesamiento por defecto
print(preprocess_string(sample_text))

# Aplicar el preprocesamiento con filtros personalizados
CUSTOM_FILTERS = [lambda x: x.lower(), strip_punctuation]
print(preprocess_string(sample_text, CUSTOM_FILTERS))

['hello', 'world', 'appl', 'weather', 'good', 'todai', 'isn']
['i', 'hello9', 'i', 'b', 'wo4rld', 'b', 'apples', 'the', 'weather', 'is', 'is', 'really', 'good', 'today', 'isn', 't', 'it']


En general, se recomienda que el procesamiento de texto se realice "on the fly" al momento de cargar cada uno de los documentos. De esta manera, se evita cargar todo el dataset en memoria.

In [7]:
class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/mycorpus.txt'):
            # Realizar el procesamiento del texto y la tokenización aquí
            yield preprocess_string(line)

corpus = MyCorpus()
for doc in corpus:
    print(doc)

['human', 'machin', 'interfac', 'lab', 'abc', 'applic']
['survei', 'user', 'opinion', 'respons', 'time']
['ep', 'user', 'interfac', 'manag']
['human', 'engin', 'test', 'ep']
['relat', 'user', 'perceiv', 'respons', 'time', 'error', 'measur']
['gener', 'random', 'binari', 'unord', 'tree']
['intersect', 'graph', 'path', 'tree']
['graph', 'minor', 'width', 'tree', 'quasi', 'order']
['graph', 'minor', 'survei']


# 3. Construir el vocabulario

La clase `corpora.Dictionary` de **gensim** nos permite construir el vocabulario a partir de los documentos del corpus, y adicionalmente asigna un ID a cada palabra del vocabulario, lo que facilita la construcción de las representaciones vectoriales que generaremos más adelante.

In [8]:
from gensim import corpora

dictionary = corpora.Dictionary(corpus)
print(dictionary)

Dictionary<31 unique tokens: ['abc', 'applic', 'human', 'interfac', 'lab']...>


Podemos guardar y cargar el diccionario construido.

In [9]:
dictionary.save('midict.dict')

In [10]:
corpora.Dictionary.load('midict.dict')
print(dictionary)

Dictionary<31 unique tokens: ['abc', 'applic', 'human', 'interfac', 'lab']...>


Ver los ids asignados a cada token del vocabulario:

In [11]:
dictionary.token2id

{'abc': 0,
 'applic': 1,
 'human': 2,
 'interfac': 3,
 'lab': 4,
 'machin': 5,
 'opinion': 6,
 'respons': 7,
 'survei': 8,
 'time': 9,
 'user': 10,
 'ep': 11,
 'manag': 12,
 'engin': 13,
 'test': 14,
 'error': 15,
 'measur': 16,
 'perceiv': 17,
 'relat': 18,
 'binari': 19,
 'gener': 20,
 'random': 21,
 'tree': 22,
 'unord': 23,
 'graph': 24,
 'intersect': 25,
 'path': 26,
 'minor': 27,
 'order': 28,
 'quasi': 29,
 'width': 30}

# 4. Representaciones vectoriales

Como lo vimos en talleres anteriores, las representaciones vectoriales son útiles para crear sistemas de recuperación de información.
**gensim** nos permite crear representaciones de bolsa de palabras, entre otras. Además, nos permite transformar representaciones de texto en otro tipo de representaciones. Por ejemplo, podemos construir una representación de bolsa de palabras con las frecuencias, y luego podemos transformar esta representación a los puntajes **tf-idf**.

## 4.1 Representacón de bolsa de palabras

In [12]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(2, 1)]


La representación vectorial generada por **gensim** consiste en tuplas, en donde el primer valor indica el id de la palabra del vocabulario, y el segundo valor representa la frecuencia de ese token en el documento.

Esta representación hace un uso eficiente de la memoria, ya que solo se generan las tuplas para los tokens con una frecuencia mayor a 0 en el documento.

**gensim** permite el uso the cualquier iterador que retorne un vector por documento a la vez.

In [13]:
class Bow_Corpus:
    def __iter__(self):
        for doc in corpus:
            # Vectorización del texto por documento
            yield dictionary.doc2bow(doc)

bow_corpus = Bow_Corpus()
for doc in bow_corpus:
    print(doc)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]
[(6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]
[(3, 1), (10, 1), (11, 1), (12, 1)]
[(2, 1), (11, 1), (13, 1), (14, 1)]
[(7, 1), (9, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1)]
[(19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]
[(22, 1), (24, 1), (25, 1), (26, 1)]
[(22, 1), (24, 1), (27, 1), (28, 1), (29, 1), (30, 1)]
[(8, 1), (24, 1), (27, 1)]


## 4.2 Representación de bolsa de palabras con puntajes **tf-idf**

En **gensim** podemos usar la representación de bolsa de palabras básica para generar representaciones más avanzadas, usando los puntajes **tf-idf** en vez de la frecuencia por token.

In [14]:
from gensim import models

# Entrenamiento del modelo TF-IDF
tfidf = models.TfidfModel(bow_corpus, normalize=True) # normalize=True normaliza los valores en el rango [0, 1]

# Aplicar el modelo TF-IDF al corpus
tfidf_corpus = tfidf[bow_corpus]

# Imnprimer los vectores TF-IDF
for doc in tfidf_corpus:
    print(doc)

[(0, 0.4500498962785324), (1, 0.4500498962785324), (2, 0.3080749612015952), (3, 0.3080749612015952), (4, 0.4500498962785324), (5, 0.4500498962785324)]
[(6, 0.6136280137156085), (7, 0.42004992797654267), (8, 0.42004992797654267), (9, 0.42004992797654267), (10, 0.30681400685780424)]
[(3, 0.4628644263314176), (10, 0.3380866887867002), (11, 0.4628644263314176), (12, 0.6761733775734003)]
[(2, 0.39942082240973076), (11, 0.39942082240973076), (13, 0.5834920793168784), (14, 0.5834920793168784)]
[(7, 0.30055933182961736), (9, 0.30055933182961736), (10, 0.21953536176370683), (15, 0.43907072352741366), (16, 0.43907072352741366), (17, 0.43907072352741366), (18, 0.43907072352741366)]
[(19, 0.48507125007266594), (20, 0.48507125007266594), (21, 0.48507125007266594), (22, 0.24253562503633297), (23, 0.48507125007266594)]
[(22, 0.31622776601683794), (24, 0.31622776601683794), (25, 0.6324555320336759), (26, 0.6324555320336759)]
[(22, 0.25098743403237606), (24, 0.25098743403237606), (27, 0.343619428161172

Llamar a `tfidf[bow_corpus]` genera un "wrapper" sobre el objeto del corpus. Por lo tanto, la real transformación vectorial se realizar por cada documento al momento de iterar spbre el dataset.

## 4.3 Guardar los modelos entrenados y el corpus transformado

Podemos guardar y cargar el modelo entrenado.

In [15]:
tfidf.save('simple_corpus.tfidf')

In [16]:
loaded_model = models.TfidfModel.load('simple_corpus.tfidf')

**gensim** tiene diferentes transformaciones disponibles. Puede ver una breve descripción de cada una [aquí](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#available-transformations).

Adicionalmente, puede guardar y cargar el corpus transformado en distintos formatos. Uno de los más populares es el formato [Market Matrix](http://math.nist.gov/MatrixMarket/formats.html)

In [17]:
corpora.MmCorpus.serialize('/tmp/corpus.mm', tfidf_corpus)

In [18]:
tfidf_corpus = corpora.MmCorpus('/tmp/corpus.mm')

In [19]:
for d in tfidf_corpus:
    print(d)

[(0, 0.4500498962785324), (1, 0.4500498962785324), (2, 0.3080749612015952), (3, 0.3080749612015952), (4, 0.4500498962785324), (5, 0.4500498962785324)]
[(6, 0.6136280137156085), (7, 0.42004992797654267), (8, 0.42004992797654267), (9, 0.42004992797654267), (10, 0.30681400685780424)]
[(3, 0.4628644263314176), (10, 0.3380866887867002), (11, 0.4628644263314176), (12, 0.6761733775734003)]
[(2, 0.39942082240973076), (11, 0.39942082240973076), (13, 0.5834920793168784), (14, 0.5834920793168784)]
[(7, 0.30055933182961736), (9, 0.30055933182961736), (10, 0.21953536176370683), (15, 0.43907072352741366), (16, 0.43907072352741366), (17, 0.43907072352741366), (18, 0.43907072352741366)]
[(19, 0.48507125007266594), (20, 0.48507125007266594), (21, 0.48507125007266594), (22, 0.24253562503633297), (23, 0.48507125007266594)]
[(22, 0.31622776601683794), (24, 0.31622776601683794), (25, 0.6324555320336759), (26, 0.6324555320336759)]
[(22, 0.25098743403237606), (24, 0.25098743403237606), (27, 0.343619428161172

## 4.4 Compatibilidad con **numpy** y **Scipy**

Si lo desea, las representaciones vectoriales de **gensim** pueden ser transformadas a areglos de **numpy**.

Recuerde que los arreglos de **numpy** son densos. Es decir, que todas sus dimensiones se representan con un valor así la mayoría de dimensiones tengan valor 0. Por lo tanto, no se recomienda esta transformación para datasets grandes ya que puede causar desbordamiento de memoria.

In [20]:
import gensim
import numpy as np

# Convertir un corpus de Gensim a una matriz densa de Numpy
numpy_matrix = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=len(dictionary))

# Convertir una matriz densa de Numpy a un corpus de Gensim
new_corpus = gensim.matutils.Dense2Corpus(numpy_matrix)

In [21]:
for doc1, doc2 in zip(new_corpus, tfidf_corpus):
    print("Son iguales las dos representaciones vectoriales:", np.allclose(doc1, doc2))
    print("Doc desde numpy", doc1)
    print("Doc original", doc2)

Son iguales las dos representaciones vectoriales: True
Doc desde numpy [(0, 0.45004990696907043), (1, 0.45004990696907043), (2, 0.308074951171875), (3, 0.308074951171875), (4, 0.45004990696907043), (5, 0.45004990696907043)]
Doc original [(0, 0.4500498962785324), (1, 0.4500498962785324), (2, 0.3080749612015952), (3, 0.3080749612015952), (4, 0.4500498962785324), (5, 0.4500498962785324)]
Son iguales las dos representaciones vectoriales: True
Doc desde numpy [(6, 0.6136280298233032), (7, 0.4200499355792999), (8, 0.4200499355792999), (9, 0.4200499355792999), (10, 0.3068140149116516)]
Doc original [(6, 0.6136280137156085), (7, 0.42004992797654267), (8, 0.42004992797654267), (9, 0.42004992797654267), (10, 0.30681400685780424)]
Son iguales las dos representaciones vectoriales: True
Doc desde numpy [(3, 0.4628644287586212), (10, 0.33808669447898865), (11, 0.4628644287586212), (12, 0.6761733889579773)]
Doc original [(3, 0.4628644263314176), (10, 0.3380866887867002), (11, 0.4628644263314176), (12

También se pueden realizar conversiones entre matrices `scipy.sparse` y **gensim**. Las representaciones `scipy.sparse` también son representaciones dispersas que son eficientes en memoria, estos objetos son usados por librerías como **scikit-learn** como salida de `CountVectorizer`.

In [22]:
# Convertir un corpus de Gensim a una matriz dispersa de Scipy
scipy_matrix = gensim.matutils.corpus2csc(tfidf_corpus)

# Convertir una matriz dispersa de Scipy a un corpus de Gensim
new_corpus = gensim.matutils.Sparse2Corpus(scipy_matrix)

In [23]:
for doc1, doc2 in zip(new_corpus, tfidf_corpus):
    print("Son iguales las dos representaciones vectoriales:", np.allclose(doc1, doc2))
    print("Doc desde scipy", doc1)
    print("Doc original", doc2)

Son iguales las dos representaciones vectoriales: True
Doc desde scipy [(0, 0.4500498962785324), (1, 0.4500498962785324), (2, 0.3080749612015952), (3, 0.3080749612015952), (4, 0.4500498962785324), (5, 0.4500498962785324)]
Doc original [(0, 0.4500498962785324), (1, 0.4500498962785324), (2, 0.3080749612015952), (3, 0.3080749612015952), (4, 0.4500498962785324), (5, 0.4500498962785324)]
Son iguales las dos representaciones vectoriales: True
Doc desde scipy [(6, 0.6136280137156085), (7, 0.42004992797654267), (8, 0.42004992797654267), (9, 0.42004992797654267), (10, 0.30681400685780424)]
Doc original [(6, 0.6136280137156085), (7, 0.42004992797654267), (8, 0.42004992797654267), (9, 0.42004992797654267), (10, 0.30681400685780424)]
Son iguales las dos representaciones vectoriales: True
Doc desde scipy [(3, 0.4628644263314176), (10, 0.3380866887867002), (11, 0.4628644263314176), (12, 0.6761733775734003)]
Doc original [(3, 0.4628644263314176), (10, 0.3380866887867002), (11, 0.4628644263314176), (1

# 5. Construcción de sistemas de recuperación

Una vez se tiene la representación vectorial de corpus definida, el corpus se puede indexar para prepararlo para realizar consultas de similitud.

In [24]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf_corpus)

Este index generado también puede guardarse y cargarse.

In [25]:
index.save('simple_corpus.index')

In [26]:
index = similarities.MatrixSimilarity.load('simple_corpus.index')

Finalmente, podemos realizar consultas a nuestro corpus.

In [27]:
# Consulta a realizar
query = 'humans complexity, engineering'

# Aplicar el mismo preprocesamiento que se aplicó al corpus
query_document = preprocess_string(query)

# Utilizar la misma representación vectorial que se utilizó en el corpus
query_bow = dictionary.doc2bow(query_document)

# Calcular la similitud de la consulta con los documentos del corpus
sims = index[tfidf[query_bow]]
print(sims)

[0.17402118 0.         0.         0.7071068  0.         0.
 0.         0.         0.        ]


Ordenar los documentos según su puntaje de similitud

In [28]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.7071068
0 0.17402118
1 0.0
2 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


# 6. Conclusiones

Librerías como **gensim** nos permiten crear sistemas de recuperación de información de una manera rápida y eficiente. Estas librerías cobran especial importancia cuando los datasets son muy grandes.